In [7]:
import os
import xmltodict
import random
import pickle
import gzip
import pandas as pd
import numpy
from pprint import pprint
from utils.tui import Progress
from utils.lexical import LexicalProcessing
from sklearn.feature_extraction.text import TfidfVectorizer

BASE_DIR = '../data/corpora/trainset'
TEST_PERCENT = 0.2
LP = LexicalProcessing()

In [2]:
def get_count():
    count = 0
    for product in os.listdir(BASE_DIR):
        corpus = {}

        product_path = '{}/{}'.format(BASE_DIR, product)
        for rank in os.listdir(product_path):  
            rank_path = '{}/{}'.format(product_path, rank)
            fls = os.listdir(rank_path)
            fls = [ x for x in fls if '.xml' in x ]
            count += len(fls)
    return count

In [3]:
def get_corpora():
    corpora_kabum = {}
    errors = []
    count = get_count()
    P = Progress(count, '')
    for product in os.listdir(BASE_DIR):
        corpus = {}

        product_path = '{}/{}'.format(BASE_DIR, product)
        for rank in os.listdir(product_path):  
            rank_path = '{}/{}'.format(product_path, rank)
            fls = os.listdir(rank_path)
            fls = [ x for x in fls if '.xml' in x ]

            reviews = []
            for fl_name in fls:
                fl_path = '{}/{}'.format(rank_path, fl_name)
                with open(fl_path) as fl_:
                    try:
                        r = xmltodict.parse(fl_.read())['review']
                        if not r['opinion']:
                            raise Exception(fl_path)
                        reviews.append(r)
                    except:
                        errors.append(fl_path)
                P.progressStep()
            corpus[rank] = reviews
        corpora_kabum[product] = corpus
    return (corpora_kabum, errors)

In [4]:
def create_dataframe(corpora_kabum):
    dataset = dict(test=[], category=[], score=[], review=[])
    for cat in corpora_kabum.keys():
        for stars in corpora_kabum[cat].keys():
            for review in corpora_kabum[cat][stars]:
                dataset['test'].append(random.choices([0,1], weights=[0.8, 0.2],k=1)[0])
                dataset['category'].append(review['category']['@value'])
                dataset['score'].append(float(review['stars']['@value']))
                dataset['review'].append(review['opinion'])
    return pd.DataFrame(data=dataset)

In [5]:
DF_PATH = 'df_kabum.pkl.gz'

def save(data, path):
    with gzip.open(path, 'wb') as f:
        f.write(pickle.dumps(data))

def load(path):
    with gzip.open(path, 'rb') as f:
        return pickle.loads(f.read())
    

In [13]:
progess_tokenize = Progress(len(df), 'lemma')
def tokenize_review(text):
    text = LP.lowercase(text)
    text = LP.remove_punctuation(text)
    tokens = LP.tokenize_words(text)
    progess_tokenize.progressStep()
    return tokens

def normalize_review(tokens):
    return ' '.join(tokens)

progess_lemma = Progress(len(df), 'lemma')
def lemmatize_words(text):
    lemmas = LP.lemmatize_sentence(text)
    progess_lemma.progressStep()
    return lemmas

In [9]:
LOAD_PICKLE = True
if LOAD_PICKLE:
    df = load(DF_PATH)
    
else:
    c_ = get_corpora()
    df = create_dataframe(c_[0])
    df['tokens'] = df.review.apply(tokenize_review)
    df['normalized'] = df.tokens.apply(normalize_review)
    df['lemmas'] = df.normalized.apply(lemmatize_words)
    save(df, DF_PATH)

df.head()

,test,category,score,review,tokens,normalized
0,0,Joystick / Controle,1.0,Comprei e já vou devolver pois nas especificaç...,"[comprei, e, já, vou, devolver, pois, nas, esp...",comprei e já vou devolver pois nas especificaç...
1,1,Joystick / Controle,1.0,Esta guitarra NÃO é compatível com Nintendo Wi...,"[esta, guitarra, não, é, compatível, com, nint...",esta guitarra não é compatível com nintendo wi...
2,1,Joystick / Controle,1.0,Não funciona corretamente no Wii. No Rockband ...,"[não, funciona, corretamente, no, wii, no, roc...",não funciona corretamente no wii no rockband o...
3,0,Joystick / Controle,1.0,"Não sei em outros consoles, mas no wii esse pr...","[não, sei, em, outros, consoles, mas, no, wii,...",não sei em outros consoles mas no wii esse pro...
4,1,Joystick / Controle,1.0,Produto nada funcional e ainda com preço exage...,"[produto, nada, funcional, e, ainda, com, preç...",produto nada funcional e ainda com preço exage...


In [14]:
save(df, DF_PATH)

In [15]:
df.head()

,test,category,score,review,tokens,normalized,lemmas
0,0,Joystick / Controle,1.0,Comprei e já vou devolver pois nas especificaç...,"[comprei, e, já, vou, devolver, pois, nas, esp...",comprei e já vou devolver pois nas especificaç...,"[comprar, e, já, ir, devolver, pois, o, especi..."
1,1,Joystick / Controle,1.0,Esta guitarra NÃO é compatível com Nintendo Wi...,"[esta, guitarra, não, é, compatível, com, nint...",esta guitarra não é compatível com nintendo wi...,"[este, guitarra, não, ser, compatível, com, ni..."
2,1,Joystick / Controle,1.0,Não funciona corretamente no Wii. No Rockband ...,"[não, funciona, corretamente, no, wii, no, roc...",não funciona corretamente no wii no rockband o...,"[não, funcionar, corretamente, o, wii, o, rock..."
3,0,Joystick / Controle,1.0,"Não sei em outros consoles, mas no wii esse pr...","[não, sei, em, outros, consoles, mas, no, wii,...",não sei em outros consoles mas no wii esse pro...,"[não, saber, em, outro, consolar, mas, o, wii,..."
4,1,Joystick / Controle,1.0,Produto nada funcional e ainda com preço exage...,"[produto, nada, funcional, e, ainda, com, preç...",produto nada funcional e ainda com preço exage...,"[produto, nado, funcional, e, ainda, com, preç..."


In [17]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('models/cbow_s50.txt')

In [19]:
ERRR = []
progess_vecs = Progress(len(df), 'vecs')
def calc_vec(tokens):
    vecs = []
    for w in tokens:
        try:
            vecs.append(model[w])
        except:
            pass
    progess_vecs.progressStep()
    if not vecs:
        ERRR.append(tokens)
        return numpy.zeros((50, ))
        
    return numpy.average(vecs, axis=0)

def aaaa(a):
    return int(a)

df['vec'] = df.lemmas.apply(calc_vec)
df['score'] = df.score.apply(aaaa)

In [21]:
train_x = df[df.test == 0].vec.values.tolist()
train_y = df[df.test == 0].score.values.tolist()

test_x = df[df.test == 1].vec.values.tolist()
test_y = df[df.test == 1].score.values.tolist()

In [22]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

clf = GaussianNB()
clf.fit(train_x, train_y)
print('Test accuracy: {:0.3f}'.format(accuracy_score(test_y, clf.predict(test_x))))
print('Train accuracy: {:0.3f}'.format(accuracy_score(train_y, clf.predict(train_x))))

Test accuracy: 0.278
Train accuracy: 0.277


In [ ]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(train_x, train_y)
print('Test accuracy: {:0.3f}'.format(accuracy_score(test_y, svc.predict(test_x))))
print('Train accuracy: {:0.3f}'.format(accuracy_score(train_y, svc.predict(train_x))))

/home/k4t0mono/.local/share/virtualenvs/ipln-LePBjxLu/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:
from sklearn.neural_network import MLPClassifier

mplc = MLPClassifier(
    hidden_layer_sizes=(100, 100),verbose=True, batch_size=200, max_iter=1000
)
mplc.fit(train_x, train_y)
print('Test accuracy: {:0.3f}'.format(accuracy_score(test_y, mplc.predict(test_x))))
print('Train accuracy: {:0.3f}'.format(accuracy_score(train_y, mplc.predict(train_x))))